In [ ]:
import pandas as pd
import glob
from os.path import basename
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline


In [ ]:
feat_labels = ['track_length_since_start', 'time_since_first_station', 'station_number', 'lat', 'lon', 'track_length', 'zeit']
X = df[feat_labels]
y = df['isadelay5']
print('SplitDataset')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

In [ ]:
datei_info = {}
df['daytype']
daytype_cats = {'Weekend': 0,  'Weekday': 1}
df['daytype_cat'] = df['daytype'].map(daytype_cats)
df['type_cat'] = pd.Categorical(df['type'], categories=df['type'].unique()).codes
df.columns

In [ ]:
print('ZeroR')
from sklearn.dummy import DummyClassifier
dummy_classifier = DummyClassifier(strategy="most_frequent")
dummy_classifier.fit( X_train,y_train)
y_dummy_classifier = dummy_classifier.predict(X_test)
datei_info['zeror'] = 1 - ((y_test != y_dummy_classifier).sum() / X_test.shape[0])
datei_info['zeror_matrix'] = confusion_matrix(y_test, y_dummy_classifier, labels=[0,1])

In [ ]:
print('DecisionTree')
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier()
model_dt = model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
datei_info['dt'] = 1 - ((y_test != y_pred_dt).sum() / X_test.shape[0])
datei_info['dt_matrix'] = confusion_matrix(y_test, y_pred_dt, labels=[0,1])
print(classification_report(y_test,y_pred_dt))

In [ ]:
print('RandomForest')
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=7,n_jobs=-1)
clf.fit(X_train,y_train)
y_pred_clf=clf.predict(X_test)
datei_info['clf'] = 1 - ((y_test != y_pred_clf).sum() / X_test.shape[0])
datei_info['clf_matrix'] = confusion_matrix(y_test, y_pred_clf, labels=[0,1])
print(classification_report(y_test,y_pred_clf))

In [ ]:
dfnew = df.dropna()
#feat_labelsnew = ['zeit',
#     'temperature_c', 'air_pressure_hpa',
 #      'relative_humidity', 'dew_point_c', 'wind_speed_kmh',
  #      'type_cat', 'time_since_last_station',
   #    'time_since_first_station', 'stay_time', 'track_length',
    #   'track_length_since_start', 'station_number', 'lon', 'lat', 'bhf_cat',
     #  'dayname_cat', 'trainno_cat', 'weather_condition_cat', 'daytype_cat']
feat_labelsnew = ['zeit', 'track_length_since_start', 'time_since_first_station','time_since_last_station', 'station_number', 'bhf_cat', 'trainno_cat']
X = dfnew[feat_labelsnew]
y = dfnew['isadelay5']
print(X.count())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)
from sklearn.ensemble import ExtraTreesClassifier
print('RandomForestnew')
from sklearn.ensemble import RandomForestClassifier
clfnew=ExtraTreesClassifier(n_estimators=len(feat_labelsnew)-2,n_jobs=-1,random_state=0)
clfnew.fit(X_train,y_train)
y_pred_clfnew=clfnew.predict_proba(X_test)[:,1]

In [ ]:
datei_info['clfnew'] = 1 - ((y_test != (y_pred_clfnew > 0.50)).sum() / X_test.shape[0])
print(y_pred_clfnew)
datei_info['clfnew_matrix'] = confusion_matrix(y_test, (y_pred_clfnew>0.50), labels=[0,1])
print(classification_report(y_test,(y_pred_clfnew > 0.5)))
print(datei_info)
for feature in zip(feat_labelsnew, clfnew.feature_importances_):
    print(feature)
print(avg(y_pred_clfnew[y_test != (y_pred_clfnew > 0.5)]))
plt.plot(y_pred_clfnew[(y_test != (y_pred_clfnew > 0.5))])

In [ ]:
import pickle
filename = 'finalized_model.sav'
# pickle.dump(clfnew, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
clfnew = pickle.load(open(filename, 'rb'))
clfnew.predict_proba([[8.0, 204.0, 3.0, 0.0, 0.0, 4.0, 4.0, 22.0]])

#datei_info['clfnew'] = 1 - ((y_test.iloc[0] != y_pred_clfnew).sum() / X_test.shape[0])
#datei_info['clfnew_matrix'] = confusion_matrix(y_test, y_pred_clfnew, labels=[0,1])
#print(classification_report(y_test,y_pred_clfnew))
#print(datei_info)
#feat_labelsnew.remove('isadelay5')
#for feature in zip(feat_labelsnew, clfnew.feature_importances_):
    #print(feature)

In [ ]:
print(X_test.iloc[0].to_list())
print(y_test.iloc[0])

In [ ]:
print(datei_info)

In [ ]:
from sklearn.feature_selection import SelectFromModel

# # Create a selector object that will use the random forest classifier to identify
# # features that have an importance of more than 0.15
sfm = SelectFromModel(clf, threshold=0.02)

# Train the selector
sfm.fit(X_train, y_train)

In [ ]:
# Print the names of the most important features
# for feature_list_index in sfm.get_support(indices=True):
#     print(feat_labels[feature_list_index])
for feature in zip(feat_labels, clf.feature_importances_):
    print(feature)
    


In [ ]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.15
#feat_labelsnew.remove('isadelay5')
sfmnew = SelectFromModel(clfnew, threshold=0.02)
# Train the selector
sfmnew.fit(X_train, y_train)

In [ ]:
# Print the names of the most important features
for feature_list_index in sfmnew.get_support(indices=True):
    print(feat_labelsnew[feature_list_index])

for feature in zip(feat_labelsnew, clfnew.feature_importances_):
    print(feature)
clfnew=RandomForestClassifier(n_estimators=len(feat_labelsnew)-2,n_jobs=-1,random_state=0)

In [ ]:
def avg(lst): 
    return sum(lst) / len(lst) 

In [ ]:
connection = [{'startbhf': 'Tübingen Hbf', 'endbhf': 'Köln Hbf', 'starttrain': 'RE_22032', 'startplatform': '2', 'endplatform': '5', 'starttime': '13:37', 'endtime': '17:05'}, {'anbhf': 'Stuttgart Hbf', 'abbhf': 'Stuttgart Hbf', 'antrain': 'RE_22032', 'abtrain': 'ICE_2536', 'transfertime': 13, 'anplatform': '2', 'abplatform': '2', 'anzeit': '14:38', 'abzeit': '14:51'}, {'anbhf': 'Mannheim Hbf', 'abbhf': 'Mannheim Hbf', 'antrain': 'ICE_2536', 'abtrain': 'ICE_106', 'transfertime': 6, 'anplatform': '3', 'abplatform': '3', 'anzeit': '15:31', 'abzeit': '15:35'}]

In [ ]:
con = connection[1]
import json
with open('newData/conversion_dicts.json') as json_file:
    conversion_dicts = json.load(json_file)

In [ ]:
con['abtrain'].startswith('IC')

In [ ]:
print(connection[2]['abtrain'])
if connection[2]['abtrain'] in conversion_dicts['trainno_cats']:
  print("blah")
else:
  print("boo")

In [ ]:
for x in range(1,len(connection)):
    data = {'zeit': 0 , 'track_length_since_start': 0 , 'time_since_first_station': 0 ,'time_since_last_station': 0 , 'station_number': 0 , 'bhf_cat': 0 , 'trainno_cat': 0 }
    con = connection[x]
    if con['antrain'] in conversion_dicts['trainno_cats']:
        if int(con['anzeit'].replace(':', ''))%100 > 30: #We dont need minutes so we form %H:%M to ~%H
            data['zeit'] = round(int(con['anzeit'].replace(':', ''))/100) + 1
        else:
            data['zeit'] = round(int(con['anzeit'].replace(':', ''))/100)
        fahrplan = pd.read_csv('newData/'+con['antrain']+'.csv', index_col=False)
        fahrplan = fahrplan.set_index('bhf')
        data['track_length_since_start'] = fahrplan.at[con['anbhf'],'track_length_since_start'][0]
        data['time_since_last_station'] = fahrplan.at[con['anbhf'],'time_since_last_station'][0]
        data['time_since_first_station'] = fahrplan.at[con['anbhf'],'time_since_first_station'][0]
        data['station_number'] = fahrplan.at[con['anbhf'],'station_number'][0]
        data['bhf_cat'] = conversion_dicts['bhf_cats'][con['anbhf']]
        data['trainno_cat'] = conversion_dicts['trainno_cats'][con['antrain']]
        print(data)
        predict(model, list(data.values()))
    else:
        print("Train not in Database")
    if con['abtrain'] in conversion_dicts['trainno_cats']:
        if int(con['abzeit'].replace(':', ''))%100 > 30: #We dont need minutes so we form %H:%M to ~%H
            data['zeit'] = round(int(con['abzeit'].replace(':', ''))/100) + 1
        else:
            data['zeit'] = round(int(con['abzeit'].replace(':', ''))/100)
        fahrplan = pd.read_csv('newData/'+con['abtrain']+'.csv', index_col=False)
        fahrplan = fahrplan.set_index('bhf')
        data['track_length_since_start'] = fahrplan.at[con['abbhf'],'track_length_since_start'][0]
        data['time_since_last_station'] = fahrplan.at[con['abbhf'],'time_since_last_station'][0]
        data['time_since_first_station'] = fahrplan.at[con['abbhf'],'time_since_first_station'][0]
        data['station_number'] = fahrplan.at[con['abbhf'],'station_number'][0]
        print(con['abbhf'])
        data['bhf_cat'] = conversion_dicts['bhf_cats'][con['abbhf']]
        data['trainno_cat'] = conversion_dicts['trainno_cats'][con['abtrain']]
        print(data)
        predict(model, list(data.values()))
    else:
        print("Train not in Database")

In [ ]:
fahrplan = pd.read_csv('newData/ICE_106.csv', index_col=False)
fahrplan = fahrplan.set_index('bhf')
#print(fahrplan.head())
fahrplan.at['Mannheim Hbf','track_length_since_start'][0]